In [148]:
import pandas as pd
import numpy as np
from jieba import analyse
import os
import jieba
from sklearn import manifold
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

# 自定义词典 & 停用词词典构建

In [238]:
##自定义携程词典
jieba.load_userdict(r"D:/Users/pjchang/Desktop/客户进线需求自动识别/stopwords-master/电话自定义词典&停用词词典/add_word.txt")

##停用词列表
g = open("D:/Users/pjchang/Desktop/客户进线需求自动识别/stopwords-master/电话自定义词典&停用词词典/cn_stopwords.txt",encoding = 'utf-8')
stopwordlist = []
for i in g.readlines():
    i = i.strip('\n')
    stopwordlist.append(i)
stopwordlist[-5:-1]

['迈克', '萧晨', '楠小星', '小昕']

# 切词 关键词top10抽取

In [224]:
datasource = pd.read_excel("D:/Users/pjchang/Desktop/客户进线需求自动识别/IM端/输入/客人进线需求_im.xlsx")
datasource[['evntid','第一通客人进线内容','员工外呼酒店内容']]

,evntid,第一通客人进线内容,员工外呼酒店内容
0,272208267,"酒店没有给您升级您到时候在联系我们哦,具体问题具体分析的,如果酒店不给您升级，您到时候联系我...",NaN
1,271733474,"您好,请问想预定哪个地区的酒店呢,我可以帮您找一下附近的酒店并告知您,嗯呐，请问是哪个机场呢...",NaN
2,272031712,"晚上好,我为您核实一下，请稍等,请问澳门消费券领券人是哪位客人,这么晚了给您添麻烦了,我现在...","好的好的，行，知道了，谢谢啊。,呃，不好意思，女士打扰您，就是我想问一下，咱们这个2号入住3..."
3,271632379,"您好,好的，这边为您联系酒店，辛苦您耐心等待一下,好的","您好，我这边携程，呃，有客人想咨询一下，如果烧烤的话，呃，酒店有提供什么东西吗？,呃，酒店是..."
4,272187369,"好的呢~,我联系酒店确认哦~会在23:55分之前在线告知您结果，辛苦您稍等一下哈~,吴先生，...",呃，多少小时之内的，那我跟客人说一下，24小时是吗？好，再见，佛山市可以接待的是吧，好，谢谢...
...,...,...,...
14601,272474118,"您好,糖糖在哒呢,请问需要糖糖处理哪些问题呢,我咨询下酒店,您是天津哪个地区哒呢,收到,我在...","330是吧，好的。,您好，唉，打扰了，女士，我想问一下就是如果明天延迟退房的话收费延迟，这种..."
14602,272391551,"麻烦请问您在青口具体位置,您好是福州，青口吗,好的,您稍等,您好，您看一下这家可以吗,请您稍...",NaN
14603,271675824,"您好，请您稍等3分钟，小敏为您查询一下订单哈～,小敏为您联系酒店咨询下哈 请稍等 15分钟...",好的好的，行，谢谢您，我这边的话跟客户说一下把您这个号码给他让客户，到时候那个到账了以后打直...
14604,272179538,"请问您是准备几号入住呢,您好，感谢您的耐心等待，浅浅核实到后台如果拆分下来预定的话是可以下单...",唉，您好，这边携程的，就客人想咨询一下您酒店这边，哦，为什么10月2号到7号的话，那个豪华大...


In [225]:
c = []
for i in range(len(datasource["第一通客人进线内容"].tolist())):
    c_stripblank = str(datasource["第一通客人进线内容"].tolist()[i]).replace(" ", "")
    c.append(c_stripblank)
e = datasource["员工外呼酒店内容"].tolist()
c

['酒店没有给您升级您到时候在联系我们哦,具体问题具体分析的,如果酒店不给您升级，您到时候联系我们，我们去帮您联系酒店,可以的',
 '您好,请问想预定哪个地区的酒店呢,我可以帮您找一下附近的酒店并告知您,嗯呐，请问是哪个机场呢,嗯呐，我会帮您查看一下并留言给您,请您放心,目前最近的有这3家酒店您可以考虑一下哦,我帮您联系酒店详细咨询一下,收到,嗯嗯,辛苦稍等30分钟,嗯呢,收到,感谢您的耐心等待,喜来登酒店告知坐巴士或打的一般20分钟左右到,1个大人三个小孩可以预定套房行政海景套房或尊贵海景套房\xa0\xa0尊贵海景套房楼层更高视野更好,目前入住没有具体要求，如果您不放心可以在入住之前再次确认避免影响您行程安排,香港嘉里酒店现在为您联系酒店核实,请稍等30分钟谢谢,收到,感谢您的耐心等待,香港嘉里酒店距离45分钟左右如果黄码和蓝码红码不可以入住，具体已政府要求为准,酒店告知房间面积不同，床的尺寸都是一样的可以加床如果当天有库存可以加床可以免费加床是折叠床95厘米*2米\xa0酒店房间只能2个大人2个小朋友（11岁及11岁以下）,喜来登也是一样的哦，目前都是没有具体要求的都是已政府公告为准的,嗯呐,因为酒店不支持红码入住，黄色和蓝色没关系哈,具体看香港政府是否有推出新得政府通知哦,嗯呐,如果您有需要可以尽快预定哦,目前酒店告知是可以得哈,因为疫情每天都会变化，不能保障后续也是这个标准哦',
 '晚上好,我为您核实一下，请稍等,请问澳门消费券领券人是哪位客人,这么晚了给您添麻烦了,我现在就为您反馈给后台核实，10分钟内回复您,我现在就加急核实,感谢您的耐心等待，我为您反馈给后台核实了，您刚才提供的图片预订的时候是有使用澳门消费券优惠的，您进线的账户，实名认证的是吾尔开西阿布都热合曼,为了更好的为您解决订单问题，请问是否方便联系您订单上预留的手机号？,收到您的图片,李女士，明天上午9.10左右致电您转接我们会员中心的同事处理,不客气的，您也早点休息哦，夜深了,不客气的，您也早点休息，我们为您核实,忙碌之余请您保重身体，希望荣荣本次服务能给您带来好的服务体验~祝您一切顺利🎈',
 '您好,好的，这边为您联系酒店，辛苦您耐心等待一下,好的',
 '好的呢~,我联系酒店确认哦~会在23:55分之前在线告知您结果，辛苦您稍等一下哈~,吴先生，小恩联系到酒店告知佛山是可以接待的

In [227]:
##切词
cut_list_c = []
for i in c :
    cutwords = jieba.lcut(str(i))
    cutwords_c = [item for item in list(cutwords) if str(item) not in stopwordlist]
    cut_list_c.append(list(set(cutwords_c)))

cut_list_e = []
for i in e :
    cutwords = jieba.lcut(str(i))
    cutwords_e = [item for item in list(cutwords) if str(item) not in stopwordlist]
    cut_list_e.append(list(set(cutwords_e)))


In [228]:
tfidf_dataset_c = []
for i in cut_list_c:
    a =(''.join(i))
    tfidf_dataset_c.append(a)

tfidf_dataset_e = []
for i in cut_list_e:
    a =(''.join(i))
    tfidf_dataset_e.append(a)


In [234]:
##TFIDF 调用两次 改tfidf_dataset后缀c&e
tfidf = analyse.extract_tags
keywords_list = []
keywords_Wlist = []
for i in tfidf_dataset_e:
    Swords_list = []
    keywords = tfidf(str(i),topK=10,withWeight=True,allowPOS=("n","f","s","ns","vn"))
    keywords_Wlist.append(keywords)
    for j in range(len(keywords)):
        Swords_list.append(keywords[j][0])
    keywords_list.append(Swords_list)
keywordsdf = pd.DataFrame(keywords_list)
keywords_Wdf = pd.DataFrame(keywords_Wlist)

#keywordsdf.columns = ['客关1','客关2','客关3','客关4','客关5','客关6','客关7','客关8','客关9','客关10']
#keywords_Wdf.columns = ['客关1w','客关2w','客关3w','客关4w','客关5w','客关6w','客关7w','客关8w','客关9w','客关10w']

keywordsdf.columns = ['员关1','员关2','员关3','员关4','员关5','员关6','员关7','员关8','员关9','员关10']
keywords_Wdf.columns = ['员关1w','员关2w','员关3w','员关4w','员关5w','员关6w','员关7w','员关8w','员关9w','员关10w']

In [213]:
"""
##textrank 调用两次 改tfidf_dataset后缀c&e
tr = analyse.textrank
keywords_list = []
keywords_Wlist = []
for i in tfidf_dataset_e:
    Swords_list = []
    keywords = tr(str(i),topK=10,withWeight=True,allowPOS=("n","f","s","ns","vn"))
    keywords_Wlist.append(keywords)
    for j in range(len(keywords)):
        Swords_list.append(keywords[j][0])
    keywords_list.append(Swords_list)
keywordsdf = pd.DataFrame(keywords_list)
keywords_Wdf = pd.DataFrame(keywords_Wlist)
##
keywordsdf.columns = ['客关1','客关2','客关3','客关4','客关5','客关6','客关7','客关8','客关9','客关10']
keywords_Wdf.columns = ['客关1w','客关2w','客关3w','客关4w','客关5w','客关6w','客关7w','客关8w','客关9w','客关10w']

#keywordsdf.columns = ['员关1','员关2','员关3','员关4','员关5','员关6','员关7','员关8','员关9','员关10']
#keywords_Wdf.columns = ['员关1w','员关2w','员关3w','员关4w','员关5w','员关6w','员关7w','员关8w','员关9w','员关10w']

In [235]:
dfresult = pd.concat([dfresult,keywordsdf], axis=1)
#dfresult = pd.concat([dfresult, keywords_Wdf], axis=1)
dfresult

,evntid,第一通客人进线内容,员工外呼酒店内容,客关1,客关2,客关3,客关4,客关5,客关6,客关7,...,员关1,员关2,员关3,员关4,员关5,员关6,员关7,员关8,员关9,员关10
0,272208267,"酒店没有给您升级您到时候在联系我们哦,具体问题具体分析的,如果酒店不给您升级，您到时候联系我...",NaN,升级,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,271733474,"您好,请问想预定哪个地区的酒店呢,我可以帮您找一下附近的酒店并告知您,嗯呐，请问是哪个机场呢...",NaN,政府公告,蓝码,黄码,红码,海景,巴士,嘉里,...,None,None,None,None,None,None,None,None,None,None
2,272031712,"晚上好,我为您核实一下，请稍等,请问澳门消费券领券人是哪位客人,这么晚了给您添麻烦了,我现在...","好的好的，行，知道了，谢谢啊。,呃，不好意思，女士打扰您，就是我想问一下，咱们这个2号入住3...",会员中心,开西,领券,手机号,澳门,核实,同事,...,离店,大床,横琴,海景,客房,None,None,None,None,None
3,271632379,"您好,好的，这边为您联系酒店，辛苦您耐心等待一下,好的","您好，我这边携程，呃，有客人想咨询一下，如果烧烤的话，呃，酒店有提供什么东西吗？,呃，酒店是...",None,None,None,None,None,None,None,...,女字,场地,收费,东西,None,None,None,None,None,None
4,272187369,"好的呢~,我联系酒店确认哦~会在23:55分之前在线告知您结果，辛苦您稍等一下哈~,吴先生，...",呃，多少小时之内的，那我跟客人说一下，24小时是吗？好，再见，佛山市可以接待的是吧，好，谢谢...,胜意,喜乐,管控,万事,核酸,疫情,None,...,旅行网,佛山市,核酸,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14601,272474118,"您好,糖糖在哒呢,请问需要糖糖处理哪些问题呢,我咨询下酒店,您是天津哪个地区哒呢,收到,我在...","330是吧，好的。,您好，唉，打扰了，女士，我想问一下就是如果明天延迟退房的话收费延迟，这种...",弹窗,天国,事事,核酸,同庆,监测,美丽,...,双床,房费,房钱,庭院,收费,体验,免费,None,None,None
14602,272391551,"麻烦请问您在青口具体位置,您好是福州，青口吗,好的,您稍等,您好，您看一下这家可以吗,请您稍...",NaN,青口,民宿,钟点,具体位置,渠道,类型,方式,...,None,None,None,None,None,None,None,None,None,None
14603,271675824,"您好，请您稍等3分钟，小敏为您查询一下订单哈～,小敏为您联系酒店咨询下哈 请稍等 15分钟...",好的好的，行，谢谢您，我这边的话跟客户说一下把您这个号码给他让客户，到时候那个到账了以后打直...,接站,微信,公共场合,西站,口罩,上车,疫情,...,接站,尾号,西站,号码,车子,司机,客户,服务,以后,None
14604,272179538,"请问您是准备几号入住呢,您好，感谢您的耐心等待，浅浅核实到后台如果拆分下来预定的话是可以下单...",唉，您好，这边携程的，就客人想咨询一下您酒店这边，哦，为什么10月2号到7号的话，那个豪华大...,花单,元花哈,大床,房态,美美,旅程,套餐,...,房态,大床,免费,None,None,None,None,None,None,None


# 客户&员工 共同高频率词

In [261]:
"""
trdfresult = pd.concat([datasource, keywordsdf], axis=1)
trdfresult = pd.concat([trdfresult, keywords_Wdf], axis=1)
trdfresult
"""

'\ntrdfresult = pd.concat([datasource, keywordsdf], axis=1)\ntrdfresult = pd.concat([trdfresult, keywords_Wdf], axis=1)\ntrdfresult\n'

In [215]:
#trdfresult.to_csv("D:/Users/pjchang/Desktop/textrank关键词_resultwith顺序.csv",encoding="utf_8_sig")

In [318]:
dfresult.to_csv("D:/Users/pjchang/Desktop/tfidf关键词_resultwith顺序1024withIM词库_IM.csv",encoding="utf_8_sig")

In [319]:
dfduplicate = dfresult[['客关1','客关2','客关3','客关4','客关5','客关6','客关7','客关8','客关9','客关10','员关1','员关2','员关3','员关4','员关5','员关6','员关7','员关8','员关9','员关10']]
extract_duplicates = []
for i in range(len(dfduplicate)):
    a = dfduplicate.iloc[i][dfduplicate.iloc[i].duplicated()][dfduplicate.iloc[i][dfduplicate.iloc[i].duplicated()].notnull()]
    extract_duplicates.append(a.tolist())
keywords_dropduplicate_df = pd.DataFrame(extract_duplicates)

In [320]:
keywords_dropduplicate_df

,0,1,2,3,4,5,6,7
0,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None
2,None,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None
4,核酸,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...
14601,None,None,None,None,None,None,None,None
14602,None,None,None,None,None,None,None,None
14603,接站,西站,车子,None,None,None,None,None
14604,None,None,None,None,None,None,None,None


In [321]:
dfwithduplicate = pd.concat([dfresult, keywords_dropduplicate_df], axis=1)
dfwithduplicate.to_csv("D:/Users/pjchang/Desktop/客户进线需求自动识别/tfidf关键词_resultwith顺序with重复词1024withIM词库_IM.csv",encoding="utf_8_sig")

# 【Tag-word】 词典读取

In [214]:
datasourceCI = pd.read_excel("D:/Users/pjchang/Desktop/客户进线需求自动识别/标签词典.xlsx",sheet_name="更新标签词库")
datasourceCI

,Unnamed: 0,Tag,0,1,2,3,4,5,6,7,...,151,152,153,154,155,156,157,158,159,160
0,0,设施信息-酒店周边设施-景点,入园,园区,套票,核实,套餐,优惠券,通票,门票,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,设施信息-酒店内部设施-餐厅,炭火,调料,食材,炉子,酒吧,晚会,餐具,午餐,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,酒店政策-儿童政策,儿童,孩子,小孩,未成年,身高,年龄,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,基础信息-酒店装修时间,装修时间,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,房型信息 - 售卖信息 - 担保,信用度,选项,担保金,银行信用,条款,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,设施信息-房型设施-洗漱用品,用品,牙刷,拖鞋,梳子,浴帽,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,房型信息-宽带/无线,密码,WIFI,宽带,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,预订查询-酒店预订,两间房,标准间,婴儿床,原单,标间,钟点,报价,无窗,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,酒店政策-疫情政策,高风险,核酸,城市,本地人,常态,外地,街道,个人信息,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,设施信息 - 房型设施 - 洗衣机,洗衣服,洗衣,洗衣房,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [217]:
C_tag = set(datasourceCI['Tag'].tolist())
C_tag = list(C_tag)

C_tag_l_all = []
for j in range(len(C_tag)):
    C_tag_lt = []
    for i in range(len(datasourceCI[datasourceCI['Tag'] == C_tag[j]])):
        dfci = (datasourceCI[datasourceCI['Tag'] == C_tag[j]].iloc[i])
        C_tag_lt += (list(dfci[2:].dropna()))
    
    C_tag_l = list(set(C_tag_lt))
    
    C_tag_l_all.append(C_tag_l)

In [220]:
DFCI = pd.DataFrame(C_tag_l_all)
DFCI.insert(0,'Tag',C_tag)

In [69]:
#DFCI.to_csv("D:/Users/pjchang/Desktop/客户进线需求自动识别/标签词典.csv",encoding="utf_8_sig")

# 判断逻辑模块

In [221]:
dict_ci = {}
for i in DFCI['Tag']:
    dict_ci [str(i)] = list(DFCI[DFCI['Tag'] == str(i)].iloc[0,1:].dropna())

In [236]:
##判断命中词组模块
def contrast(l,diction):
    result = []
    for key in diction.keys():
        lj = (set(l)&set(diction[key]))
        if len(list(lj)) != 0:
            result.append(key)
    result1 = list(set(result))
    return result1

def special_rule(tag_list):
    if "预订查询-酒店预订" in tag_list:
        #tag_list.remove("酒店政策-入住政策-取消政策")
        tag_list = ["预订查询-酒店预订"]
    return tag_list

r_list = []
for i in range(len(dfresult)):
    l1 = list(dfresult.iloc[i].dropna())
    a = contrast(l1,dict_ci)
    b = special_rule(a)
    r_list.append(b)
len(r_list)

14606

In [237]:
df_finalresult = pd.concat([datasource[['evntid','第一通客人进线内容','员工外呼酒店内容']], pd.DataFrame(r_list)], axis=1)
df_finalresult.to_csv("D:/Users/pjchang/Desktop/客户进线需求自动识别/IM端/IM_判断结果.csv",encoding="utf_8_sig")